In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-rna-3d-folding/sample_submission.csv
/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv
/kaggle/input/stanford-rna-3d-folding/test_sequences.csv
/kaggle/input/stanford-rna-3d-folding/train_labels.v2.csv
/kaggle/input/stanford-rna-3d-folding/validation_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_sequences.csv
/kaggle/input/stanford-rna-3d-folding/train_sequences.v2.csv
/kaggle/input/stanford-rna-3d-folding/MSA/R1108.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/8EVR_EC.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/1ZDI_S.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/5FJ1_H.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/2NBY_A.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/3DEG_J.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/2M21_A.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/8U3M_A.MSA.fasta
/kaggle/input/stanford-rna-3d-folding/MSA/2OM7_F.MSA.fas

In [2]:
import pandas as pd

input_path = "/kaggle/input/stanford-rna-3d-folding/test_sequences.csv"

df = pd.read_csv(input_path)

print(df.head())


  target_id                                           sequence  \
0     R1107  GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...   
1     R1108  GGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUU...   
2     R1116  CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGG...   
3   R1117v2                     UUGGGUUCCCUCACCCCAAUCAUAAAAAGG   
4     R1126  GGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUU...   

  temporal_cutoff                                        description  \
0      2022-05-28  CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ri...   
1      2022-05-27  CPEB3 ribozyme\nChimpanzee\nChimpanzee CPEB3 H...   
2      2022-06-04  Cloverleaf RNA\nPoliovirus\nCrystal Structure ...   
3      2022-06-03  PreQ1 class I type III riboswitch\nK. pneumoni...   
4      2022-06-11  Traptamer\nSynthetic\nAdditional Information: ...   

                                       all_sequences  
0  >7QR4_1|Chain A|U1 small nuclear ribonucleopro...  
1  >7QR3_1|Chains A, B|U1 small nuclear ribonucle...  
2  >8

In [3]:
import pandas as pd
import numpy as np

# Load test data
input_path = "/kaggle/input/stanford-rna-3d-folding/test_sequences.csv"
df = pd.read_csv(input_path)

submission_rows = []

# Spiral parameters
num_turns = 3
radius = 10
z_step = 1.5

for idx, row in df.iterrows():
    target_id = row['target_id']
    sequence = row['sequence']
    L = len(sequence)

    theta = np.linspace(0, num_turns * 2 * np.pi, L)
    z = np.linspace(0, L * z_step, L)
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)

    for i in range(L):
        res_id = i + 1
        resname = sequence[i]
        coords = []
        for _ in range(5):
            coords.extend([
                x[i] + np.random.normal(0, 0.2),
                y[i] + np.random.normal(0, 0.2),
                z[i] + np.random.normal(0, 0.2)
            ])
        submission_rows.append([f"{target_id}_{res_id}", resname, res_id] + coords)

# Create output DataFrame
columns = ['ID', 'resname', 'resid'] + [f'{axis}_{i}' for i in range(1,6) for axis in ['x','y','z']]
submission_df = pd.DataFrame(submission_rows, columns=columns)

# Save
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

# Preview
print(submission_df.head())


        ID resname  resid       x_1       y_1       z_1        x_2       y_2  \
0  R1107_1       G      1  9.780936 -0.038480 -0.099695  10.216707  0.191331   
1  R1107_2       G      2  9.584668  2.889041  1.805316   9.815632  2.782085   
2  R1107_3       G      3  8.450164  4.952218  3.580424   8.714443  5.102561   
3  R1107_4       G      4  6.622384  7.719029  4.536216   6.920726  7.272618   
4  R1107_5       G      5  4.160908  8.898337  6.169485   5.123407  9.184667   

        z_2       x_3       y_3       z_3        x_4       y_4       z_4  \
0 -0.107649  9.898392 -0.079175  0.096937  10.248157  0.092717 -0.022665   
1  1.231456  9.521588  2.862798  1.250161   9.628879  2.649373  1.485805   
2  2.748038  8.601273  5.172596  3.033224   8.346123  5.329676  3.129088   
3  4.280066  6.758294  7.878078  4.462474   6.610046  7.387371  4.699779   
4  6.081502  4.223663  8.816664  6.155335   4.298124  8.653137  6.093302   

         x_5       y_5       z_5  
0  10.057995  0.074761  0.3